# <center><font color=blue>Ẩn dữ liệu và chia sẻ thông tin
# <center><font color=blue>Đồ án môn học </font>
# <center>ALASKA2 Image Steganalysis
### <center>*Detect secret data hidden within digital images*</center>


## Giới thiệu thành viên và phân công công công việc:



| Tên | MSSV | Nhiệm vụ | Hoàn thành |
|:-------|-------|:-------|-------|
| Nguyễn Trí Đức | 20120060 | Tiền xử lý dữ liệu, report | 100% |
| Nguyễn Thế Anh | 20120248 | Tìm hiểu và xây dựng mô hình, report | 100% |


## Giới thiệu chung về đồ án:

Đồ án dựa trên cuộc thi [ALASKA2 Image Steganalysis](https://www.kaggle.com/competitions/alaska2-image-steganalysis) trên <font color=blue>__Kaggle__</font>. Những tệp hình ảnh tưởng chừng như "vô hại" có thể chứa đựng thông điệp ẩn được nhúng một cách tinh vi, khiến mắt thường khó lòng phát hiện. Machine learning là một công cụ quan trọng trong việc nhận diện những dữ liệu bí mật này.

Trong đồ án này, chúng ta sẽ cần tạo ra một phương pháp hiệu quả và đáng tin cậy để phát hiện dữ liệu bí mật ẩn trong các hình ảnh kỹ thuật số tưởng chừng như vô hại. Thay vì giới hạn nguồn dữ liệu, những hình ảnh này đã được thu thập với 50 máy ảnh khác nhau (từ điện thoại thông minh đến máy ảnh cao cấp có định dạng đầy đủ) và được xử lý theo nhiều kiểu khác nhau. Phương pháp thành công sẽ bao gồm các thuật toán phát hiện mạnh mẽ với tỷ lệ sai sót tối thiểu. Cụ thể, là tạo ra một mô hình học máy phù hợp với độ chính xác cao để phân tích và tính toán xem các bức ảnh trong folder test có bị nhúng dữ liệu mật hay không.

## Mô tả Dataset:
- Bộ dữ liệu này chứa một số lượng lớn các hình ảnh jpeg gốc gọi là "Cover" và các hình ảnh tương tự nhưng đã bị nhúng dữ liệu ẩn bằng các thuật toán steganography gồm JMiPOD, JUNIWARD, UERD. Mục tiêu của cuộc thi là xác định hình ảnh nào trong tập Test có thông điệp ẩn được nhúng.


- Lưu ý rằng để làm cho cuộc thi thực tế hơn, độ dài của các thông điệp ẩn (payload) sẽ không được cung cấp. Thông tin duy nhất có sẵn trên tập kiểm tra là:
    - Mỗi thuật toán nhúng được sử dụng với cùng một xác suất.
    - Payload (độ dài thông điệp) được điều chỉnh sao cho "độ khó" là xấp xỉ như nhau bất kể nội dung của hình ảnh. Các hình ảnh với nội dung mượt mà sẽ được sử dụng để ẩn các thông điệp ngắn hơn trong khi các hình ảnh có kết cấu cao sẽ được sử dụng để ẩn nhiều bit bí mật hơn. Payload được điều chỉnh theo cách tương tự cho cả tập kiểm tra và tập huấn luyện.
    - Độ dài thông điệp trung bình là 0.4 bit mỗi hệ số AC DCT khác không.
    - Các hình ảnh đều được nén với một trong ba hệ số chất lượng JPEG sau: 95, 90 hoặc 75.
    
    
- __Tập tin__:
    - Cover/ chứa 75k hình ảnh gốc, dùng để huấn luyện.
    - JMiPOD/ chứa 75k ví dụ về thuật toán JMiPOD được áp dụng lên các hình ảnh cover.
    - JUNIWARD/ chứa 75k ví dụ về thuật toán JUNIWARD được áp dụng lên các hình ảnh cover.
    - UERD/ chứa 75k ví dụ về thuật toán UERD được áp dụng lên các hình ảnh cover.
    - Test/ chứa 5k hình ảnh của tập kiểm tra. Đây là các hình ảnh mà bạn cần dự đoán.
    - sample_submission.csv chứa một ví dụ về bài nộp trong định dạng chính xác.

## Bài toán chi tiết:

## Cách tính điểm trong cuộc thi [ALASKA2 Image Steganalysis](https://www.kaggle.com/competitions/alaska2-image-steganalysis):

- __Giới thiệu về Đường cong ROC và AUC__:
    - Đường cong ROC (Receiver Operating Characteristic) là một công cụ quan trọng trong đánh giá hiệu suất của các mô hình phân loại, biểu diễn khả năng phân loại các lớp của mô hình. Đường cong này thể hiện mối quan hệ giữa Tỷ Lệ Dương Tính Thực Sự (TPR - True Positive Rate) và Tỷ Lệ Dương Tính Giả (FPR - False Positive Rate) tại các ngưỡng khác nhau.
    - Đặc điểm của đường cong ROC:
        - Đường cong ROC càng gần góc trên bên trái của biểu đồ (điểm [0,1], nơi TPR = 1 và FPR = 0) thì hiệu suất của mô hình càng cao.
        - Một đường cong đi từ góc dưới bên trái ([0,0]) đến góc trên bên phải ([1,1]) của biểu đồ, thể hiện mức độ phân biệt giữa các lớp của mô hình.
        - Đây là ví dụ về đường cong ROC với ngưỡng 0.0 - 0.5 - 1.0 : [ROC Example](https://miro.medium.com/v2/resize:fit:1200/1*Bgc9QOjhnL70g2SQxyj6hQ.png)
- __Khái niệm AUC__: AUC là diện tích dưới đường cong ROC (Receiver Operating Characteristic), đánh giá tổng thể hiệu quả phân loại của mô hình, với AUC = 1 chỉ ra hiệu quả tuyệt đối và AUC = 0.5 tương đương với dự đoán ngẫu nhiên.


- __Trọng số của các vùng trên ROC curve__: 
    - Trong cuộc thi này, để nhấn mạnh vào độ chính xác của việc phát hiện với tỉ lệ báo động sai thấp, AUC có trọng số (weighted AUC) được sử dụng. Các vùng khác nhau của ROC curve được tính trọng số theo các tham số đã chọn:
        - `tpr_thresholds = [0.0, 0.4, 1.0]` - true positive rate được chia thành 3 vùng 0.0; 0.4 và 1.0
        - `weights = [2, 1]` - trọng số từ 0 đến 0.4 là 2 và trọng số từ 0.4 đến 1 là 1
    - Điều này có nghĩa là diện tích giữa TPR 0 và 0.4 được tính trọng số là 2, và diện tích giữa 0.4 và 1 được tính trọng số là 1.
    
    
- __Công thức tính__:
    - Đầu tiên, chia ROC curve thành các vùng dựa trên các ngưỡng TPR đã cho.
    - Sau đó, tính toán diện tích dưới đường cong ROC trong mỗi vùng.
    - Diện tích của mỗi vùng được nhân với trọng số tương ứng.
    - Tổng diện tích có trọng số được chuẩn hóa bằng tổng của các trọng số ( 2 + 1 = 3 ) để đảm bảo rằng AUC cuối cùng nằm trong khoảng từ 0 đến 1.
    - Ví dụ : 
        - Gọi $S_1$ là phần diện tích dưới đường cong ROC của vùng 1 ứng với trọng số $W_1$.
        - $S_2$ là phần diện tích dưới đường cong của vùng 2 ứng với trọng số $W_1$.
        - Ta có công thức tính weighted AUC:


<center>$wAUC = \frac{W_1.S_1 + W_2.S_2}{W_1+W_2} = \frac{2.S_1 + S_2}{3}$</center>


- __Nhận xét__: $S_1$ sẽ ảnh hưởng đến wAUC nhiều hơn $S_2$, hay nói cách khác diện tích ở vùng FPR thấp càng lớn thể hiện khả năng phát hiện tin ẩn đáng tin cậy hơn và tỉ lệ báo động sai sẽ thấp hơn, đó cũng chính là mục tiêu của cuộc thi này.

## Mô tả bài toán:

1. Chuẩn bị dữ liệu:
    - Dữ liệu được chuẩn bị thông qua việc đọc các hình ảnh từ tập dữ liệu. Các hình ảnh được đọc, chuyển đổi từ BGR sang RGB, chuẩn hóa (chỉ số pixel được chia cho 255 để chuyển giá trị pixel về khoảng [0, 1]), và cuối cùng áp dụng các biến đổi.
    - Tiền xử lý dữ liệu:
        - Chuyển đổi hình ảnh sang định dạng tensor phù hợp với mô hình Pytorch.
        - Áp dụng các phép biến đổi dữ liệu (như xoay, cắt, thu nhỏ, v.v.) để tăng cường dữ liệu và cải thiện khả năng tổng quát của mô hình.
2. Tải Mô hình và Trọng số:
    - Một mô hình học máy đã được huấn luyện trước đó (có thể là một mạng nơ-ron sâu) được tải với trọng số từ một tệp checkpoint. Điều này cho phép mô hình sử dụng kiến thức đã học từ quá trình huấn luyện trước đó.
3. Dự đoán:
    - Mô hình được sử dụng để thực hiện dự đoán trên tập dữ liệu kiểm tra. Dự đoán này được thực hiện bằng cách đưa hình ảnh qua mô hình để nhận đầu ra là xác suất của việc hình ảnh có bị nhúng dữ liệu hay không.
4. Tính Xác suất:
    - Đầu ra của mô hình (thường là logits) được chuyển qua hàm softmax để tính xác suất của mỗi lớp. Trong trường hợp này, xác suất của việc hình ảnh không chứa dữ liệu nhúng được tính toán, và xác suất của việc hình ảnh chứa dữ liệu nhúng được lấy là phần bù của xác suất này (1 - xác suất hình ảnh không chứa dữ liệu nhúng).
5. Ghi Kết quả:
    - Kết quả dự đoán, bao gồm tên hình ảnh và xác suất của việc hình ảnh chứa dữ liệu nhúng, được ghi vào một tệp CSV. Tệp này sau đó có thể được sử dụng để nộp lên các nền tảng đánh giá, như Kaggle.

## Model EfficientNet - Pytorch:

- __PyTorch__ là một thư viện mã nguồn mở phổ biến được sử dụng cho học máy, đặc biệt là trong lĩnh vực xử lý ảnh và thị giác máy tính. Nó cung cấp một tập hợp các công cụ và API mạnh mẽ để xây dựng, huấn luyện và triển khai các mô hình học sâu. PyTorch đóng vai trò quan trọng trong việc phát triển và sử dụng EfficientNet, nó cung cấp nền tảng và các công cụ cần thiết để xây dựng, huấn luyện và triển khai EfficientNet một cách hiệu quả.


- __EfficientNet__ là một họ mô hình mạng neural sâu được phát triển bởi Google AI, được công bố lần đầu vào năm 2019. EfficientNet được thiết kế để đạt được hiệu suất cao nhất có thể về cả độ chính xác và tốc độ tính toán bằng cách sử dụng phương pháp mở rộng đồng thời độ sâu (depth), độ rộng (width) và độ phân giải (resolution) của mô hình. Một số đặc điểm chính của EfficientNet bao gồm:
    - Compound Scaling: EfficientNet sử dụng một phương pháp mở rộng gọi là compound scaling, mở rộng đồng thời độ sâu (depth), độ rộng (width) và độ phân giải (resolution) của mạng neural theo một cách cân bằng và tối ưu.
    - Baseline Model - EfficientNet-B0: EfficientNet bắt đầu với một mô hình cơ bản nhỏ gọi là EfficientNet-B0, được xây dựng bằng cách sử dụng kỹ thuật neural architecture search (NAS). Sau đó, các phiên bản lớn hơn (B1 đến B7) được tạo ra bằng cách mở rộng mô hình B0 theo phương pháp compound scaling.
    - Hiệu suất cao: EfficientNet đạt được state-of-the-art accuracy trên nhiều tác vụ nhận dạng ảnh và cạnh tranh tốt về hiệu quả tính toán so với các mô hình khác như ResNet hay DenseNet.
    - EfficientNet chủ yếu được sử dụng trong các tác vụ liên quan đến hình ảnh. Đầu vào thường là các hình ảnh với các kích thước khác nhau. Để đưa vào mô hình, các hình ảnh này thường được tiền xử lý như thay đổi kích thước, chuẩn hóa và các phép biến đổi khác (augmentation) để tăng tính đa dạng của dữ liệu huấn luyện.
    - Đầu ra của mô hình EfficientNet thường là một vector xác suất cho mỗi lớp trong bài toán phân loại. Ví dụ, nếu bạn đang phân loại hình ảnh thành 4 loại khác nhau, đầu ra sẽ là một vector với 4 giá trị, mỗi giá trị biểu thị xác suất của hình ảnh thuộc về một lớp cụ thể.
    
    
- __Trong đồ án này, EfficientNet có nhiệm vụ__: nhận diện và phân loại các hình ảnh dựa trên khả năng chứa thông tin ẩn, cách áp dụng:
    - Đầu vào: Sử dụng hình ảnh từ tập dữ liệu, với mỗi hình ảnh được đánh dấu là "bị nhúng dữ liệu" hoặc "không bị nhúng dữ liệu".
    - Mô hình EfficientNet: Chọn phiên bản phù hợp của EfficientNet (ví dụ: EfficientNet-B0 đến EfficientNet-B7) phù hợp với quy mô và yêu cầu tính toán của dự án.
    - Tiền xử lý: Các hình ảnh được tiền xử lý trước khi đưa vào mô hình, bao gồm chuẩn hóa, thay đổi kích thước và các biến đổi khác để tăng tính đa dạng và khả năng tổng quát hóa của mô hình.
    - Huấn luyện và đánh giá: Huấn luyện mô hình EfficientNet trên tập dữ liệu huấn luyện để học các đặc trưng của hình ảnh có chứa dữ liệu ẩn. Sau đó, đánh giá mô hình trên tập dữ liệu kiểm định để đo lường độ chính xác và hiệu suất của mô hình.
    - Các bước cụ thể:
        - Sử dụng EfficientNet để trích xuất đặc trưng từ hình ảnh.
        - Áp dụng hàm mất mát phù hợp (ví dụ: LabelSmoothing) để huấn luyện mô hình.
        - Đánh giá mô hình bằng các phép đo như AUC để đánh giá khả năng phân loại của mô hình.

In [2]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=88417a385e2b30298eedf295b2074ad6704cb695e90d8879cd0eb7962467f176
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet-pytorch


### Cài đặt thư viện EfficientNet - Pytorch:
- Để có thể sử dụng mô hình `EfficientNet` được đào tạo cho phân loại ảnh 

PATH csv file: /content/drive/MyDrive/Alaska/train_75.csv
'alaska2-image-steganalysis''train-75'

In [3]:
# library
import torch
import os
import numpy as np
import pandas as pd
import cv2
from torch import nn
from efficientnet_pytorch import EfficientNet
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from glob import glob
from torch.utils.data import DataLoader
from torch.nn import functional as F
from sklearn import metrics

from glob import glob
import random
from sklearn.model_selection import GroupKFold

from datetime import datetime
import time
import warnings

from catalyst.data.sampler import BalanceClassSampler
from torch.utils.data.sampler import SequentialSampler


### Import những thư viện cần thiết
Những thư viện cần thiết cho xử lý dữ liệu, mô hình mạng nơ-ron, tối ưu hóa, đánh giá hiệu quả của mô hình:
- *__torch__*: thư viện `PyTorch`, dùng để xây dựng và huấn luyện các mô hình học sâu;


- *__pandas__*: thao tác và phân tích dữ liệu dạng bảng (`DataFrame`);


- *__cv2__*: thư viện `OpenCV`, dùng để xử lý hình ảnh và video;


- *__albumentations__*: tăng cường dữ liệu (data augmentation) cho các ảnh;


- *__ToTensorV2__*: chuyển đổi ảnh thành `tensor`, định dạng cần thiết cho `PyTorch`;


- *__metrics__*: module `metrics` từ `scikit-learn`, chứa các hàm để đánh giá mô hình học máy;


- *__GroupKFold__*: import lớp `GroupKFold` từ `scikit-learn`, một biến thể của *k-fold cross-validation* đảm bảo cùng một nhóm không được biểu diễn trong cả tập huấn luyện và kiểm tra.

In [ ]:
'''
Run on local to make train.csv file
'''

# from glob import glob
# import random
# import pandas as pd
# from sklearn.model_selection import GroupKFold


# def group_k_fold():
#     dataset = []


#     for label, kind in enumerate(["Cover", "JMiPOD", "JUNIWARD", "UERD"]):
#         count = 0 
#         for path in glob("../input/alaska2-image-steganalysis/Cover/*.jpg"):
#             dataset.append(
#                 {"kind": kind, "image_name": path.split("\\")[-1], "label": label}
#             )

#             count += 1
#     print(count)

#     random.shuffle(dataset)
#     dataset = pd.DataFrame(dataset)


#     gkf = GroupKFold(n_splits=5) # Chia data thanh 5 fold, 
                                    # moi fold co du 4 file .jpg cua 4 folder
                                # Chon 1 fold làm valid data, 4 fold con lai la train data

#     dataset.loc[:, "fold"] = 0
#     print(dataset)
#     for fold_number, (train_index, val_index) in enumerate(
#         gkf.split(X=dataset.index, y=dataset["label"], groups=dataset["image_name"])
#     ):
#         dataset.loc[dataset.iloc[val_index].index, "fold"] = fold_number


#     # save to /input/metadata/train_75.csv
#     dataset.to_csv("../input/metadata/train_75.csv", index=False)

# group_k_fold()  


### Tạo file 'train_75.csv' :
* Chay dưới local để tạo;


* Chứa thông tin *metadata* của các hình ảnh trong bộ dữ liệu, gồm nhãn, loại ảnh, thông tin về fold để phục vụ cho việc huấn luyện mô hình với kỹ thuật *k-fold cross-validation*. 


* Về *k-fold cross-validation*:
    * Là 1 kỹ thuật phổ biến để đánh giá hiệu suất của mô hình học máy. Nó hoạt động bằng cách chia tập dữ liệu thành *k* phần (*folds*) và sử dụng từng phần để luân phiên làm tập huấn luyện và tập kiểm tra. Quá trình này được lặp lại *k* lần, đảm bảo rằng mỗi phần dữ liệu được sử dụng một lần làm tập kiểm tra.   
    
    * Cách thực hiện:
        * Chia tập dữ liệu thành *k* phần (*folds*) có kích thước bằng nhau - trong đồ án này chúng ta sẽ chia `k = 5`;
        * Với mỗi nhóm: chọn nhóm hiện tại làm tập kiểm tra (`valid_dataset`) và *k-1* nhóm còn lại làm tập huấn luyện (`train_dataset`) ==> huấn luyện mô hình học máy trên tập huấn luyện ==> đánh giá hiệu suất mô hình trên tập kiểm tra;
        *  Tính toán hiệu suất trung bình của mô hình trên mỗi nhóm.    
        
    * Ưu điểm: 
        * Ước tính độ chính xác mô hình một cách khách quan, giúp giảm thiểu ảnh hưởng của việc chia dữ liệu ngẫu nhiên, cung cấp kết quả đánh giá mô hình ổn định và đáng tin cậy.
        * Khi mô hình *overfitting* với tập huấn luyện, nó có thể hoạt động kém trên dữ liệu mới. *K-fold CV* có thể giúp phát hiện tình trạng *overfitting* bằng cách theo dõi hiệu suất mô hình trên các tập kiểm tra khác nhau.


* Giải thích chi tiết:
    * Tạo mảng `dataset` lưu trữ thông tin các mẫu dữ liệu
    * Duyệt qua 4 folders *Cover, JMiPOD, JUNIWARD, UERD* và lặp qua tất quả file *.jpg* của mỗi folder;
    * Với mỗi file tạo 1 từ điển gồm `kind` (loại), `image_name` (tên) và `label` (nhãn) và `append` vào mảng `dataset`;
    * Dùng `shuffle` để trộn ngẫu nhiên danh sách `dataset` nhằm đảm bảo mô hình được huấn luyện một cách hiệu quả và có tính tổng quát tốt, giúp tránh hiện tượng *overfitting*, không phụ thuộc vào thứ tự của dữ liệu.
    * Sau đó tiến hành khởi tạo `gkf - GroupKFold` là chia dữ liệu thành 5 folds, mỗi fold sẽ có 4 loại ảnh (*Cover, JMiPOD, JUNIWARD, UERD*) từ 4 thư mục khác nhau và đảm bảo rằng cùng một nhóm không xuất hiện trong cả tập huấn luyện và tập kiểm thử. Một fold được chọn làm dữ liệu kiểm thử (valid data), và 4 fold còn lại là dữ liệu huấn luyện (train data). 
    * Gán `fold` cho mỗi mẫu bằng cách duyệt qua các `fold` được tạo và gán giá trị fold tương ứng cho mỗi mẫu trong `DataFrame` `dataset`.
    * Cuối cùng, `DataFrame` `dataset` được lưu vào một file `CSV - train_75.CSV`, cho phép lưu trữ và sử dụng lại dữ liệu đã được chia fold một cách dễ dàng.

In [8]:
# constant
MODEL = "efficientnet-b2"
CSV_DIR = "/kaggle/input/train-75/" # Thu mục chứa csv thông tin metadata input
DATA_ROOT_PATH = "/kaggle/input/alaska2-image-steganalysis" # thư mục chứa data của đề bài
OUTPUT_DIR = "/kaggle/working/submission.csv"
data = pd.read_csv('/kaggle/input/train-75/train_75.csv')
data

,kind,image_name,label,fold
0,JUNIWARD,20402.jpg,2,1
1,Cover,28617.jpg,0,4
2,JUNIWARD,07694.jpg,2,1
3,JUNIWARD,51201.jpg,2,3
4,JMiPOD,03417.jpg,1,0
...,...,...,...,...
299995,JUNIWARD,52099.jpg,2,0
299996,UERD,50987.jpg,3,1
299997,JMiPOD,69068.jpg,1,4
299998,UERD,18377.jpg,3,2


### Constant: 


- Thiết lập các hằng số là các đường dẫn quan trọng;
- Đọc dữ liệu từ file `train-75.csv` vừa tạo.


In [9]:
'''
config.py
'''
class TrainGlobalConfig:
    csv_file = "train_75.csv"  # can change to train_90.csv or train_95.csv
    fold_number = 2
    num_workers = 4
    batch_size = 16
    n_epochs = 2 # so spoch huan luyen, 2 vi khong co time
    lr = 2e-4

    verbose = True # kiem soat viec in thong bao khi huan luyen
    verbose_step = 1

    step_scheduler = True  # cap nhat ti le hoc tap sau moi buoc toi uu hoa
    valid_scheduler = False  # khong cap nhat ti le hoc tap trong qua trinh validation

    SchedulerClass = torch.optim.lr_scheduler.OneCycleLR
    scheduler_params = dict(
        max_lr=lr,
        epochs=n_epochs,
        steps_per_epoch=None,
        pct_start=0.1,
        anneal_strategy="cos",
        cycle_momentum=True,
        div_factor=10.0,
    )
print(TrainGlobalConfig.csv_file)

train_75.csv


### Config: 


- Cài đặt lớp `TrainGlobalConfig` gồm một số cấu hình cơ bản, định nghĩa các biến toàn cục như:
    - `csv_file`, `fold_number` (chọn fold làm tập `valid_dataset` để test), `lr` (learning rate - kiểm soát mức độ mô hình cập nhật trọng số của nó dựa trên các lỗi trong quá trình training), `verbose` (kiểm soát in thông báo trong khi huấn luyện), `step_scheduler` và `valid_scheduler` (kiểm soát việc cập nhật learning rate ), ...
    - `scheduler_params`: từ điển chứa các tham số về việc cập nhật tỷ lệ học tập trong quá trình training.

In [10]:
'''
net.py
'''
def get_net():
    net = EfficientNet.from_pretrained(MODEL)
    net._fc = nn.Linear(in_features=1408, out_features=4, bias=True)
    return net


### `get_net()`:
- Tạo mô hình mạng nơ-ron sử dụng `EfficientNet` đã được huấn luyện trước (`pretrained`);
- Thay đổi lớp fully connected (FC) cuối cùng với 1408 đặc trưng đầu vào và 4 đầu ra để phù hợp với tác vụ phân loại vào 4 lớp.

In [11]:
'''
utils.py
'''
def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


###  `seed_everything(seed)`:


- Đặt giá trị seed cố định cho tất cả các thư viện và môi trường có thể ảnh hưởng đến tính ngẫu nhiên trong quá trình chạy chương trình như `random`, biến môi trường `PYTHONHASHSEED`, thư viện` NumPy`, thư viện `PyTorch`.
- Giúp đảm bảo tính nhất quán, đảm bảo rằng kết quả của chương trình là có thể tái tạo được

In [7]:
'''
dataset.py
DATA_ROOT_PATH : /kaggle/input/alaska2-image-steganalysis
'''

class Dataset:
    def __init__(self, df, num_classes=4, transforms=None):
        super().__init__()
        self.df = df
        self.num_classes = num_classes
        self.transforms = transforms

    def __getitem__(self, index):
        # image la data anh index.jpg cua thu muc "kind"
        # target la vector co dang [x, x, x, x] xac dinh loai anh
        filename = (self.df["kind"].values)[index] + "/" + (self.df["image_name"].values)[index] 
        image = cv2.imread(f"{DATA_ROOT_PATH}/{filename}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {"image": image}
            sample = self.transforms(**sample)
            image = sample["image"]

        label_idx = (self.df["label"].values)[index]
        target = self.onehot(self.num_classes, label_idx) 
        return image, target

    def __len__(self):
        return len(self.df)

    def get_labels(self):
        return list(self.df["label"].values)

    def onehot(self, num_classes, target):
        vec = torch.zeros(num_classes, dtype=torch.float32)
        vec[target] = 1.0
        return vec


class TestDataset:
    def __init__(self, image_names, transforms=None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image = cv2.imread(f"{DATA_ROOT_PATH}/Test/{image_name}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {"image": image}
            sample = self.transforms(**sample)
            image = sample["image"]

        return image_name, image

    def __len__(self):
        return self.image_names.shape[0]


### `class Dataset` và `class TestDataset`:


- Xử lý dữ liệu huấn luyện (có nhãn) và dữ liệu kiểm tra (không có nhãn);
- Giải thích chi tiết:


- `Class Dataset`:
    - Phương thức khởi tạo `__init__`:  `transforms` - các biến đổi áp dụng cho ảnh nhằm chuẩn hóa, tăng cường dữ liệu.
    - Phương thức lấy mẫu dựa vào index `__getitem__(self, index)`: 
        - Từ `index` xác định tên file và đường dẫn tương ứng, chuyển ảnh từ BGR thành RGB do OpenCV đọc hình ảnh theo thứ tự BGR (Blue, Green, Red) thay vì thứ tự RGB (Red, Green, Blue),  chuẩn hóa giá trị pixel về [0, 1] (Giá trị pixel ban đầu thường nằm trong phạm vi từ 0 đến 255. Chuẩn hóa giá trị pixel về phạm vi từ 0 đến 1 giúp quá trình học của mô hình học máy diễn ra ổn định và nhanh chóng hơn. → image /= 255.0), sau đó áp dụng các phương thức `transform`, tạo nhãn dưới dạng vector one-hot dựa vào nhãn của ảnh.
        - Trả về cặp ảnh và nhãn tương ứng.
    - Phương thức lấy độ dài dataset `__len__`;
    - Phương thức lấy danh sách nhãn `getlabels`;
    - Phương thức chuyển đổi nhãn sang dạng one-hot `onehot`:
        - Tạo một vector zeros với kích thước bằng số lớp [x,x,x,x];
        - Đặt giá trị 1.0 tại vị trí tương ứng với nhãn của ảnh. (vd: [0, 0, 1.0, 0] )
    - Tạo nhãn với vecctor one-hot (dành cho các bài toán với số lớp lớn hơn 2) nhằm: 
        - Mỗi lớp được biểu diễn rõ ràng và độc lập bằng một vector mà trong đó chỉ có một phần tử là 1 và tất cả các phần tử khác là 0
        - Tương thích với các hàm tính mất mát - loss function như hàm mất mát `cross-entropy` yêu cầu đầu vào là dạng one-hot
        - Tránh việc khiến mô hình hiểu lầm khi sử dụng nhãn số nguyên (0, 1, 2, 3) là nhãn có số cao hơn thì quan trọng hơn
        - Giúp việc cập nhật trọng số trong quá trình học của mô hình trở nên hiệu quả hơn.
        
        
- `Class TestDataset`:
    - Các phương thức tương tự Dataset nhưng TestDataset không cần nhãn vì nó được thiết kế để đánh giá hiệu suất của mô hình sau khi mô hình đã được huấn luyện.
    - Trong quá trình kiểm tra, mô hình sẽ dự đoán nhãn cho các mẫu trong tập kiểm tra và các dự đoán này sẽ được so sánh với nhãn thực tế để đánh giá độ chính xác, độ nhạy và các chỉ số khác. Vì thế, trong lớp `TestDataset`, nhãn không cần thiết vì nhiệm vụ của nó chỉ là cung cấp dữ liệu đầu vào cho mô hình. 
    - Nếu nhãn được sử dụng trong tập kiểm tra, có nguy cơ mô hình có thể "nhìn thấy" hoặc sử dụng nhãn trong quá trình dự đoán, điều này sẽ dẫn đến rò rỉ dữ liệu và kết quả đánh giá sẽ không chính xác. Bằng cách không bao gồm nhãn trong `TestDataset`, chúng ta đảm bảo rằng mô hình chỉ dựa vào dữ liệu đầu vào để đưa ra dự đoán.

In [8]:
'''
transform.py
'''
def get_augs():
    train_augs = albumentations.Compose(
        [
            albumentations.Resize(height=512, width=512, p=1.0),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.RandomRotate90(p=0.5),
            ToTensorV2(p=1.0),
        ],
        p=1.0, # p = 1.0 means that the transform will be applied to all images
    )

    valid_augs = albumentations.Compose(
        [
            albumentations.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ],
        p=1.0,
    )

    return train_augs, valid_augs


### `get_augs()`:
- Giúp tạo ra các phép biến đổi cần thiết để chuẩn bị dữ liệu cho quá trình huấn luyện và kiểm tra mô hình. Các phép biến đổi này bao gồm thay đổi kích thước ảnh, lật ảnh, xoay ảnh và chuyển đổi ảnh thành tensor. Sử dụng các phép biến đổi này giúp tăng cường dữ liệu (Data Augmentation), cải thiện khả năng tổng quát hóa, đảm bảo tính chính xác của mô hình và chuẩn bị dữ liệu ở định dạng phù hợp cho quá trình huấn luyện với PyTorch.


- Trả về hai đối tượng albumentations.Compose, mỗi đối tượng chứa một chuỗi các phép biến đổi (augmentation) ảnh được áp dụng cho tập huấn luyện (train) và tập xác thực (valid). Các phép biến đổi này được thực hiện bằng thư viện albumentations, một thư viện phổ biến cho việc tiền xử lý và tăng cường dữ liệu ảnh.


- Đối với dữ liệu huấn luyện `train_augs`:
    - Thực hiện các phép biến đổi như thay đổi kích thước, lật ngang, lật dọc, xoay ảnh 90 độ, chuyển đổi ảnh thành tensor PyTorch để phù hợp với đầu vào của mô hình. 
    - Tỷ lệ thực hiện `p = 1.0` tương đương 100% tức là toàn bộ ảnh đều thực hiện biến đổi hay `p = 0.5` tương đương 50%
    
    
- Đối với dữ liệu huấn luyện `valid_augs`:
    - Chỉ thực hiện thay đổi kích thước và chuyển đổi ảnh thành tensor PyTorch. Các biến đổi này ít đa dạng hơn so với quá trình huấn luyện để đảm bảo tính chính xác và khách quan khi đánh giá mô hình.

In [12]:
'''
loss.py
'''
class LabelSmoothing(nn.Module):
    def __init__(self, smoothing=0.05):
        super().__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing

    def forward(self, logits, targets):
        if self.training:
            logits = logits.float()
            targets = targets.float()

            log_probs = F.log_softmax(logits, dim=-1)

            nll_loss = (-log_probs * targets).sum(-1)
            smooth_loss = -log_probs.mean(dim=-1)

            loss = self.confidence * nll_loss + self.smoothing * smooth_loss

            return loss.mean()

        else:
            return F.cross_entropy(logits, targets)


### Tính toán loss:
- Định nghĩa lớp `LabelSmoothing` kế thừa từ lớp `nn.Module` của `Pytorch` => thực hiện kỹ thuật làm mịn nhãn, nằm trong 1 phần của hàm tính mất mát - loss function trong quá trình huấn luyện mô hình.
- __Ý tưởng__: Label Smoothing là một kỹ thuật giúp giảm hiện tượng quá mức tự tin (overconfidence) của mô hình bằng cách điều chỉnh nhãn mục tiêu, thay vì sử dụng nhãn "cứng" (0 hoặc 1) trong vector one-hot, vd: (0, 0, 1, 0), nhãn được làm mềm bằng cách phân bổ một lượng nhỏ độ tin cậy vào các lớp khác, vd: (0.0125, 0.0125; 0.9625; 0.0125)
- __Công thức__: kĩ thuật làm mượt label có dạng one-hot theo công thức sau:
<center>\[ q_i = \begin{cases} 
1 - \varepsilon & \text{if } i = y, \\
\frac{\varepsilon}{K - 1} & \text{otherwise},
\end{cases} \]</center>

với K là số class (*K = 4*) và ε   là một hằng số nhỏ do ta chọn . Kĩ thuật này làm giảm sự tự tin thái quá của model, làm giảm khả năng overfit.
- Phương thức khởi tạo `__init__(self, smoothing = 0,05)` với độ làm mịn nhãn `smoothing = 0,05`, độ tin cậy của nhãn thực `confidence = 1.0 - smoothing`.


- Phương thức `forward(self, logits, targets)`: phương thức chính để tính mất mát. Việc làm mềm nhãn (`targets`) không được thực hiện trực tiếp mà là thông qua cách tính toán mất mát để giả lập việc làm mềm nhãn, bằng cách phân bổ một lượng nhỏ độ tin cậy đến tất cả các lớp, bao gồm cả lớp đúng và các lớp sai, vd: (0, 0, 1, 0) => (0.0125, 0.0125; 0.9625; 0.0125). Điều này giúp giảm bớt sự tự tin quá mức của mô hình vào nhãn thực tế và khuyến khích mô hình phân bổ một lượng nhỏ xác suất vào các lớp khác, qua đó làm mềm nhãn một cách gián tiếp.

    - Nếu đang trong giai đoạn huấn luyện (`self.training`):
        - Chuyển `logits` (đầu ra của mô hình trước khi áp dụng softmax) và `targets` (nhãn thực tế) sang kiểu `float`
        - Tính log của softmax của `logits` để chuyển đổi `logits` thành `log-probabilities`
        - Tính toán `nll_loss` (negative log likelihood loss) - tính mất mát đối với các nhãn thực tế
        - Tính toán `smooth_loss` - tính bằng cách lấy trung bình của `log-probabilities` 
        - Tính tổng hợp mất mát - `loss`: kết hợp `nll_loss` và `smooth_loss` với trọng số được xác định bởi `confidence` và `smoothing`
        - Trả về trung bình của tổng mất mát trên tất cả các mẫu. 
        
    - Nếu đang trong quá trình kiểm tra: hàm mất mát sẽ sử dụng `F.cross_entropy` trực tiếp trên `logits` và `targets` mà không áp dụng làm mịn nhãn.

In [1]:
'''
metric.py
'''
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def alaska_weighted_auc(y_true, y_valid):
    """
    https://www.kaggle.com/anokas/weighted-auc-metric-updated
    """
    tpr_thresholds = [0.0, 0.4, 1.0]
    weights = [2, 1]

    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_valid, pos_label=1)

    # size of subsets
    areas = np.array(tpr_thresholds[1:]) - np.array(tpr_thresholds[:-1])

    # The total area is normalized by the sum of weights such that the final weighted AUC is between 0 and 1.
    normalization = np.dot(areas, weights)

    competition_metric = 0
    for idx, weight in enumerate(weights):
        y_min = tpr_thresholds[idx]
        y_max = tpr_thresholds[idx + 1]
        mask = (y_min < tpr) & (tpr < y_max)

        if sum(mask) != 0:

            x_padding = np.linspace(fpr[mask][-1], 1, 100)

            x = np.concatenate([fpr[mask], x_padding])
            y = np.concatenate([tpr[mask], [y_max] * len(x_padding)])
            y = y - y_min  # normalize such that curve starts at y=0
            score = metrics.auc(x, y)

        else:
            score = 1.0

        submetric = score * weight
        # best_subscore = (y_max - y_min) * weight
        competition_metric += submetric

    return competition_metric / normalization


class RocAucMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.y_true = np.array([0, 1])
        self.y_pred = np.array([0.5, 0.5])
        self.score = 0

    def update(self, y_pred, y_true):
        y_true = y_true.cpu().numpy().argmax(axis=1).clip(min=0, max=1).astype(int)
        y_pred = 1 - nn.functional.softmax(y_pred, dim=1).data.cpu().numpy()[:, 0]
        self.y_true = np.hstack((self.y_true, y_true))
        self.y_pred = np.hstack((self.y_pred, y_pred))
        self.score = alaska_weighted_auc(self.y_true, self.y_pred)

    @property
    def avg(self):
        return self.score


### Lớp `AverageMeter(object)':
- Tính toán và lưu giá trị trung bình với các phương thức `__init__(self)`, `reset(self)`, `update(self, val, n =1)` - cập nhật giá trị trung bình.

### Hàm `alaska_weighted_auc(y_true, y_valid)`:


- Hàm tính toán AUC có trọng số, một biến thể của AUC (Area Under the Curve) dành riêng cho bài toán phát hiện ẩn dữ liệu của cuộc thi <font color = blue> __Alaska2__</font>  trên <font color=blue> __Kaggle__</font>, với các phạm vi khác nhau của `TPR` (True Positive Rate) có mức độ quan trọng khác nhau cho bài toán.
- __Bước 1: Định nghĩa ngưỡng và trọng số:__
    - `tpr_thresholds` định nghĩa các ngưỡng cho TPR. Trong trường hợp này, có ba ngưỡng: 0.0, 0.4, và 1.0.
    - `weights` định nghĩa trọng số cho mỗi phạm vi giữa các ngưỡng TPR. Có hai trọng số: 2 cho phạm vi từ 0.0 đến 0.4 và 1 cho phạm vi từ 0.4 đến 1.0.
- __Bước 2: Tính toán ROC Curve__
    - Sử dụng `metrics.roc_curve` để tính toán FPR (False Positive Rate), TPR, và ngưỡng từ y_true (nhãn thực) và y_valid (xác suất dự đoán là dương tính).
- __Bước 3: Chuẩn bị cho việc tính toán AUC có trọng số__
    - Tính `areas` là diện tích của mỗi phạm vi giữa các ngưỡng TPR.
    - Tính `normalization` là tổng trọng số của các phạm vi, được sử dụng để chuẩn hóa AUC về [0, 1].
- __Bước 4: Tính toán AUC có trọng số__
    - Lặp qua mỗi phạm vi giữa các ngưỡng TPR:
        - Tính `mask` để xác định các điểm nằm trong phạm vi hiện tại.
        - Nếu có điểm trong phạm vi:
            - Tính `x_padding` để đảm bảo rằng đường cong ROC kết thúc tại FPR = 1.
            - Tạo `x` và `y` bằng cách kết hợp các giá trị FPR và TPR với `x_padding` và giá trị `y_max` tương ứng.
            - Chuẩn hóa `y` để đường cong bắt đầu từ 0.
            - Tính `score` là AUC của phạm vi hiện tại.
        - Nếu không có điểm nào trong phạm vi, gán `score` là 1.0.
        - Tính `submetric` là AUC có trọng số cho phạm vi hiện tại.
    - Cộng dồn `submetric` vào `competition_metric`.
- __Bước 5: Trả về AUC có trọng số chuẩn hóa__
    - Chia `competition_metric` cho `normalization` để đảm bảo giá trị cuối cùng nằm trong khoảng từ 0 đến 1.
    
### Lớp `RocAucMeter`:
- Theo dõi và tính toán AUC (Area Under the Curve) có trọng số cho một tập dữ liệu dự đoán và nhãn thực tế trong quá trình huấn luyện hoặc kiểm thử mô hình.
- Phương thức `__init__(self)`: gọi phương thức `reset` để thiết lập các giá trị ban đầu.
- Phương thức `reset(self)`: 
    - Đặt lại `y_true` và `y_pred` thành mảng numpy với giá trị khởi tạo là `[0, 1]` và `[0.5, 0.5]` tương ứng, đảm bảo rằng cả hai mảng đều có ít nhất một giá trị dương và một giá trị âm, tránh lỗi khi tính toán AUC.
    - `score` được đặt lại thành 0, chuẩn bị cho việc tính toán mới.
- Phương thức `update(self, y_pred, y_true)`: cập nhật giá trị dự đoán `y_pred` và giá trị thực `y_true` sau đó tính toán AUC có trọng số dùng hàm `alaska_weighted_auc`.
    - `y_true` được chuyển thành mảng `numpy`, sử dụng `argmax` để chọn lớp có xác suất cao nhất từ đầu ra của mô hình, sau đó giới hạn giá trị trong khoảng [0, 1] và chuyển thành kiểu int.
    - `y_pred` được xử lý để lấy xác suất của lớp 0 (giả sử lớp 0 là lớp quan tâm) bằng cách sử dụng hàm `softmax`, sau đó lấy `1 - `giá trị đó để phản ánh xác suất của lớp đối diện.
    - Cập nhật `self.y_true` và `self.y_pred` bằng cách ghép mảng mới vào mảng hiện tại.
    - Tính toán `score` mới sử dụng hàm `alaska_weighted_auc` với `self.y_true` và `self.y_pred` cập nhật.
- Thuộc tính `avg`: Một property Python, trả về giá trị `score` hiện tại, cho phép truy cập dễ dàng và đọc được giá trị AUC có trọng số mà không cần gọi một phương thức cụ thể.


- __Hàm softmax__:
    - Hàm softmax là một hàm kích hoạt được sử dụng chủ yếu ở lớp đầu ra của các mô hình phân loại đa lớp. Nó chuyển đổi một véc-tơ của giá trị thô (còn gọi là logits) từ lớp đầu ra của mô hình thành một véc-tơ của giá trị xác suất có tổng bằng 1. Cụ thể, hàm softmax đảm bảo rằng mỗi giá trị đầu ra nằm trong khoảng (0, 1) và tổng của tất cả các giá trị đầu ra bằng 1, làm cho chúng có thể được giải thích như là xác suất.
    - Tác dụng của hàm Softmax:
        - Chuyển đổi Logits thành Xác suất: Hàm softmax chuyển đổi các giá trị đầu ra của mô hình (logits) thành xác suất bằng cách sử dụng công thức toán học đặc biệt, giúp dễ dàng giải thích kết quả đầu ra của mô hình.
        - So sánh giữa các Lớp: Bằng cách chuyển đổi đầu ra thành xác suất, softmax giúp so sánh trực tiếp mức độ "tự tin" của mô hình đối với mỗi lớp, làm cho việc lựa chọn lớp dự đoán cuối cùng trở nên dễ dàng hơn.
        - Tối Ưu Hóa Dễ Dàng: Khi kết hợp với hàm mất mát như cross-entropy, việc sử dụng softmax giúp quá trình tối ưu hóa (học) của mô hình trở nên hiệu quả hơn, vì gradient của hàm mất mát có thể được tính toán một cách chính xác và ổn định.

In [10]:
'''
learner.py
'''
warnings.filterwarnings("ignore")


class Learner:
    def __init__(self, model, config, base_dir="./"):
        self.model = model.cuda()
        self.config = config

        self.base_dir = base_dir
        self.log_path = f"{self.base_dir}/log.txt"
        self.best_loss = 1e5

        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=config.lr)
        self.scheduler = config.SchedulerClass(
            self.optimizer, **config.scheduler_params
        )
        self.criterion = LabelSmoothing().cuda()
        self.log("Learner prepared.")

    def fit(self, train_loader, valid_loader):
        for epoch in range(self.config.n_epochs):
            if self.config.verbose:
                timestamp = datetime.utcnow().isoformat()
                self.log(f"\n{timestamp}\n")

            # Training
            t = time.time()
            train_loss, auc_scores = self.train(train_loader)

            self.log(
                f"[RESULT]: Train. Epoch: {epoch}, train_loss: {train_loss.avg:.5f}, auc_score: {auc_scores.avg:.5f}, time: {(time.time() - t):.5f}"
            )
            self.save(f"{self.base_dir}/last-checkpoint.bin")

            # Validation
            t = time.time()
            valid_loss, auc_scores = self.validation(valid_loader)

            self.log(
                f"[RESULT]: Val. Epoch: {epoch}, valid_loss: {valid_loss.avg:.5f}, auc_score: {auc_scores.avg:.5f}, time: {(time.time() - t):.5f}"
            )
            if valid_loss.avg < self.best_loss:
                self.best_loss = valid_loss.avg
                self.save(
                    f"{self.base_dir}/fold{self.config.fold_number}-checkpoint-{str(epoch).zfill(3)}epoch.bin"
                )
                for path in sorted(
                    glob(
                        f"{self.base_dir}/fold{self.config.fold_number}-checkpoint-*epoch.bin"
                    )
                )[:-3]:
                    os.remove(path)

            if self.config.valid_scheduler:
                self.scheduler.step(metrics=valid_loss.avg)

    def train(self, train_loader):

        self.model.train()

        train_loss = AverageMeter()
        auc_scores = RocAucMeter()

        t = time.time()
        for step, (images, targets) in enumerate(train_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    lr = self.optimizer.param_groups[0]["lr"]
                    print(
                        f"Train step {step}/{len(train_loader)}, Learning rate = {1e6*lr:.6f}e-6, "
                        + f"Train loss: {train_loss.avg:.5f}, AUC score: {auc_scores.avg:.5f}, "
                        + f"Time: {(time.time() - t):.5f}",
                        end="\r",
                    )

            images = images.cuda().float()
            targets = targets.cuda().float()

            self.optimizer.zero_grad()
            preds = self.model(images)
            loss = self.criterion(preds, targets)
            loss.backward()
            self.optimizer.step()

            if self.config.step_scheduler:
                self.scheduler.step()

            auc_scores.update(preds, targets)
            batch_size = images.shape[0]
            train_loss.update(loss.detach().item(), batch_size)

        return train_loss, auc_scores

    def validation(self, valid_loader):

        self.model.eval()

        valid_loss = AverageMeter()
        auc_scores = RocAucMeter()

        t = time.time()
        for step, (images, targets) in enumerate(valid_loader):
            if self.config.verbose:
                if step % self.config.verbose_step == 0:
                    print(
                        f"Validation step {step}/{len(valid_loader)}, "
                        + f"Valid loss: {valid_loss.avg:.5f}, AUC score: {auc_scores.avg:.5f}, "
                        + f"Time: {(time.time() - t):.5f}",
                        end="\r",
                    )

            with torch.no_grad():
                images = images.cuda().float()
                targets = targets.cuda().float()
                preds = self.model(images)
                loss = self.criterion(preds, targets)

                auc_scores.update(preds, targets)
                batch_size = images.shape[0]
                valid_loss.update(loss.detach().item(), batch_size)

        return valid_loss, auc_scores

    def save(self, path):
        self.model.eval()
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "scheduler_state_dict": self.scheduler.state_dict(),
                "best_loss": self.best_loss,
            },
            path,
        )

    def load(self, path):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        self.best_loss = checkpoint["best_loss"]

    def log(self, message):
        if self.config.verbose:
            print(message)
        with open(self.log_path, "a+") as logger:
            logger.write(f"{message}\n")


### Lớp `Learner`:
- Huấn luyện và đánh giá một mô hình học sâu với PyTorch, gồm các phương thức:

- Phương thức khởi tạo `__init__(self, model, config, base_dir="./")`:
    - `model.cuda()`: chuyển mô hình sang thiết bị CUDA (GPU) để tăng tốc độ tính toán.
    - `self.config`: lưu cấu hình huấn luyện.
    - `self.base_dir`: đường dẫn lưu các tệp liên quan
    - `self.log_path`: đường dẫn đến tệp log.
    - `self.best_loss`: khởi tạo giá trị best loss với một số lớn (1e5).
    - `self.optimizer`: Khởi tạo optimizer AdamW với các tham số của mô hình và learning rate từ cấu hình.
    - `self.scheduler`: Khởi tạo scheduler dựa trên lớp và tham số từ cấu hình.
    - `self.criterion`: Khởi tạo hàm mất mát Label Smoothing và đưa nó lên GPU.
    - `self.log("Learner prepared.")`: Ghi log rằng Learner đã được chuẩn bị.
    
    
- Phương thức `fit(self, train_loader, valid_loader)`: huấn luyện và đánh giá mô hình cho từng epoch
    - Lặp qua mỗi epoch:
        - Kiểm tra verbose: nếu bật, in ra timestamp bắt đầu epoch.
        - Huấn luyện:
            - Gọi hàm `train` (được định nghĩa ở dưới) để huấn luyện trên tập `train_loader`, nhận về `train_loss` và `auc_scores`.
            - Ghi log kết quả huấn luyện (mất mát trung bình, AUC score trung bình, thời gian).
            - Lưu checkpoint mô hình với tên `last-checkpoint.bin`.
        - Đánh giá:
            - Gọi hàm `validation` để đánh giá trên `valid_loader`.
            - Ghi log kết quả đánh giá (mất mát trung bình, AUC score trung bình, thời gian).
            - Kiểm tra nếu mất mát đánh giá trung bình nhỏ hơn `best_loss`:
                - Cập nhật `best_loss`.
                - Lưu checkpoint mô hình với tên theo fold, epoch (ví dụ: `fold{fold_number}-checkpoint-{epoch}-epoch.bin)`.
                - Xóa các checkpoint cũ hơn (giữ lại 3 checkpoint gần nhất).
            - Kiểm tra `valid_scheduler`: nếu có, điều chỉnh learning rate của scheduler dựa trên mất mát đánh giá trung bình.


- Phương thức `train(self, train_loader)`: thực hiện quá trình huấn luyện mô hình bằng cách lặp qua tập dữ liệu, tính toán mất mát, cập nhật mô hình dựa trên mất mát này, và theo dõi hiệu suất huấn luyện qua mất mát và điểm số AUC.
    - Huấn luyện mô hình trên bộ dữ liệu `train_loader`.
    - Chuyển mô hình sang chế độ huấn luyện (`model.train()`).
    - Khởi tạo các object `AverageMeter` để theo dõi mất mát trung bình và `RocAucMeter` để theo dõi AUC score trung bình.
    - Lặp qua các batch dữ liệu:
        - Kiểm tra `verbose`: nếu bật và đủ điều kiện (`step % verbose_step == 0`), in ra thông tin về learning rate, mất mát trung bình, AUC score trung bình, thời gian.
        - Chuyển dữ liệu hình ảnh và nhãn - `images` và `targets` được chuyển lên GPU (nếu có) và chuyển đổi sang dạng số thực.
        - Tính toán Mất mát và Cập nhật Mô hình:
            - Xóa gradient cũ bằng `self.optimizer.zero_grad()`.
            - Dự đoán (`preds`) từ mô hình dựa trên `images`. (`preds = self.model(images)`).
            - Tính toán mất mát (`loss`) bằng hàm `criterion` giữa dự đoán và mục tiêu thực tế.
            - Thực hiện backpropagation - lan truyền ngược gradient (`loss.backward()`).
            - Cập nhật trọng số mô hình (`optimizer.step()`).
        - Kiểm tra `step_scheduler`: nếu có, điều chỉnh learning rate của scheduler.
        - Cập nhật `auc_scores` và `train_loss` với kết quả của batch hiện tại.
        - Cuối cùng, phương thức trả về `train_loss` và `auc_scores` để theo dõi hiệu suất huấn luyện.
       
        
- Phương thức `validation(self, valid_loader)`: thực hiện quá trình đánh giá mô hình trên tập dữ liệu kiểm định bằng cách lặp qua tập dữ liệu, tính toán mất mát mà không cập nhật mô hình, và theo dõi hiệu suất kiểm định qua mất mát và điểm số AUC.
    - Đánh giá mô hình trên bộ dữ liệu `valid_loader`.
    - Chuyển mô hình sang chế độ đánh giá (`model.eval()`).
    - Khởi tạo các object `AverageMeter` để theo dõi mất mát trung bình và `RocAucMete`r để theo dõi AUC score trung bình.
    - Lặp qua các batch dữ liệu: 
        - Kiểm tra `verbose`: nếu bật và đủ điều kiện (`step % verbose_step == 0`), in ra thông tin về mất mát trung bình, AUC score trung bình, thời gian.
        - Xử lý Dữ liệu và Tính toán Mất mát Mà Không Cập nhật Mô hình:
            - Sử dụng `torch.no_grad()` để vô hiệu hóa việc tính toán gradient, giảm bớt việc sử dụng bộ nhớ và tăng tốc độ tính toán.
            - Dữ liệu `images` và `targets` được chuyển lên GPU (nếu có) và chuyển đổi sang dạng số thực.
            - Dự đoán (`preds`) từ mô hình dựa trên `images`.
            - Tính toán mất mát giữa dự đoán và mục tiêu thực tế.
       - Cập nhật `auc_scores` và `valid_loss` với kết quả từ batch hiện tại.
       - Cuối cùng, phương thức trả về `valid_loss` và `auc_scores` để theo dõi hiệu suất kiểm định.
           
           
- Phương thức `save(self, path)`: lưu trạng thái hiện tại của quá trình huấn luyện vào một file theo đường dẫn được cung cấp (`path`), gồm: 
    - self.model.state_dict(): Trạng thái của mô hình, bao gồm các giá trị trọng số và bias của tất cả các lớp trong mô hình.
    - self.optimizer.state_dict(): Trạng thái của trình tối ưu hóa, lưu trữ các thông tin như learning rate hiện tại và các tham số khác liên quan đến quá trình cập nhật trọng số.
    - self.scheduler.state_dict(): Trạng thái của trình lập lịch learning rate, lưu các thông tin về learning rate hiện tại và lịch trình thay đổi learning rate.
    - self.best_loss: Giá trị mất mát xác nhận tốt nhất đạt được trong quá trình huấn luyện tính đến thời điểm lưu.
    
    
- Phương thức `load(self, path)`:  nạp trạng thái huấn luyện được lưu trước đó từ một file theo đường dẫn được cung cấp (`path`) và phục hồi các thành phần của quá trình huấn luyện.


- Phương thức `log(self, message)`: Ghi log thông tin - ghi một thông báo (`message`) vào file log.
    - Kiểm tra thuộc tính `verbose` trong `self.config`: nếu là True, thông báo sẽ được in ra console.
    - Ghi thông báo vào file `self.log_path` ở chế độ mở để thêm (`"a+"`). Điều này cho phép ghi nhiều thông báo vào cùng một file log theo thứ tự thời gian.

In [11]:
'''
train.py
'''

def train():
    SEED = 42
    seed_everything(SEED)
    csv_file = TrainGlobalConfig.csv_file
    df = pd.read_csv(CSV_DIR + csv_file)

    train_augs, valid_augs = get_augs()
    train_dataset = Dataset(
        df=df[df["fold"] != TrainGlobalConfig.fold_number], # Lấy hàng có giá trị fold khác fold_number trong config
        transforms=train_augs,
    )
            
    valid_dataset = Dataset(
        df=df[df["fold"] == TrainGlobalConfig.fold_number], # ngược lại
        transforms=valid_augs,
    )
    
    train_loader = DataLoader(
        train_dataset,
        sampler=BalanceClassSampler(
            labels=train_dataset.get_labels(), mode="downsampling"
        ),
        batch_size=TrainGlobalConfig.batch_size,
        pin_memory=False,
        drop_last=True,
        num_workers=TrainGlobalConfig.num_workers,
    )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=TrainGlobalConfig.batch_size,
        num_workers=TrainGlobalConfig.num_workers,
        shuffle=False,
        sampler=SequentialSampler(valid_dataset),
        pin_memory=False,
    )
    TrainGlobalConfig.scheduler_params["steps_per_epoch"] = (
        len(train_dataset) // TrainGlobalConfig.batch_size
    )

    net = get_net().cuda()
    learner = Learner(model=net, config=TrainGlobalConfig)
    learner.fit(train_loader, valid_loader)

### hàm `train()`: 
- Mô tả quy trình huấn luyện một mô hình học máy, từ việc chuẩn bị dữ liệu đến việc khởi tạo và huấn luyện mô hình, cụ thể:
- Khởi tạo Seed: Đặt seed (`SEED = 42`) để đảm bảo tính nhất quán và tái lập được của quá trình huấn luyện.
- Đọc Dữ liệu: Sử dụng `pandas` để đọc dữ liệu từ một tệp CSV, đường dẫn được xác định bởi `TrainGlobalConfig.csv_file` và `CSV_DIR`.
- Chuẩn bị Dữ liệu và Augmentation:
    - `get_augs()` trả về các phép biến đổi (augmentations) cho dữ liệu huấn luyện và kiểm định.
    - Tạo hai tập dữ liệu, `train_dataset` và `valid_dataset`, dựa trên giá trị của cột `fold`. Dữ liệu được chia theo giá trị `fold` để phục vụ cho quá trình kiểm định chéo (cross-validation).
- Khởi tạo DataLoader:
    - `train_loader` sử dụng `BalanceClassSampler` để cân bằng số lượng mẫu giữa các lớp thông qua downsampling, giúp giảm thiểu vấn đề mất cân bằng lớp.
    - `valid_loader` không cần cân bằng lớp và sử dụng `SequentialSampler` để duyệt qua dữ liệu kiểm định theo thứ tự.
- Cấu hình `Scheduler`: Cập nhật `scheduler_params` trong `TrainGlobalConfig` bằng cách tính toán số bước mỗi epoch dựa trên kích thước của tập dữ liệu huấn luyện và kích thước batch.
- Khởi tạo Mô hình và Learner:
    - `get_net()` tạo mô hình và chuyển nó lên GPU .
    - `Learner` được khởi tạo với mô hình và cấu hình huấn luyện. `Learner` là một lớp trừu tượng hóa quá trình huấn luyện, bao gồm việc theo dõi mất mát, tối ưu hóa, và lập lịch tốc độ học.
- Huấn luyện Mô hình: Cuối cùng, sử dụng phương thức fit của `Learner` để bắt đầu quá trình huấn luyện mô hình với `train_loader` và `valid_loader`.

In [ ]:
train()

In [ ]:
'''
inference.py
Chạy với data test và ghi auc vào submission.csv
'''
# Thay doi file .bin neu co cai tot hon
CHECKPOINT_PATH = "/kaggle/working/last-checkpoint.bin"

def list_files_in_dir(path):
    # Ensure the path ends with a slash
    if not path.endswith('/'):
        path += '/'
    
    # Use glob to list all files in the directory
    files = glob(path + '*')
    
    return files

files = list_files_in_dir(CHECKPOINT_PATH)
print(files)

class DatasetSubmissionRetriever:
    def __init__(self, image_names, transforms=None):
        super().__init__()
        self.image_names = image_names
        self.transforms = transforms

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image = cv2.imread(f"{DATA_ROOT_PATH}/Test/{image_name}", cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {"image": image}
            sample = self.transforms(**sample)
            image = sample["image"]

        return image_name, image

    def __len__(self):
        return self.image_names.shape[0]


def submiss_run():
    _, valid_augs = get_augs()
    test_dataset = DatasetSubmissionRetriever(
        image_names=np.array(
            [
                path.split("/")[-1]
                for path in glob("/kaggle/input/alaska2-image-steganalysis/Test/*.jpg")
            ]
        ),
        transforms=valid_augs,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=2,
        drop_last=False,
    )

    checkpoint = torch.load(CHECKPOINT_PATH)
    net = get_net()
    net.load_state_dict(checkpoint["model_state_dict"])
    net = net.cuda()
    result = {"Id": [], "Label": []}
    for step, (image_names, images) in enumerate(test_loader):
        print(step, end="\r")

        y_pred = net(images.cuda())
        y_pred = (
            1 - F.softmax(y_pred, dim=1).data.cpu().numpy()[:, 0]
        )  # first column corresponds to 'proba of no hidden code'

        result["Id"].extend(image_names)
        result["Label"].extend(y_pred)

    submission = pd.DataFrame(result)
    submission.to_csv(OUTPUT_DIR, index=False)


### Inference:
- Thực hiện quy trình dự đoán trên tập dữ liệu kiểm tra, sử dụng một mô hình đã được huấn luyện và lưu trữ trong một checkpoint, và sau đó ghi kết quả dự đoán vào một tệp CSV (`submission.csv`)
- `CHECKPOINT_PATH` chứa đường dẫn đến tệp checkpoint mà từ đó mô hình sẽ được tải.
- `list_files_in_dir` là một hàm để liệt kê tất cả các tệp trong một thư mục chỉ định.
- `DatasetSubmissionRetriever` là một lớp dataset được tạo để xử lý việc tải và biến đổi hình ảnh từ tập dữ liệu kiểm tra.
- Hình ảnh được đọc và chuyển đổi từ BGR sang RGB, sau đó được chuẩn hóa bằng cách chia cho 255.0. Nếu có biến đổi (`transforms`) được cung cấp, chúng sẽ được áp dụng cho hình ảnh
- Tạo `test_dataset` sử dụng `DatasetSubmissionRetriever` với danh sách tên hình ảnh được lấy từ thư mục kiểm tra.
- `test_loader` được tạo để tải dữ liệu kiểm tra với kích thước batch, số lượng worker, và cài đặt khác.
- Tải Checkpoint và Mô hình:
    - Tải checkpoint từ đường dẫn đã định nghĩa.
    - Khởi tạo mô hình và tải trạng thái mô hình từ checkpoint.
    - Chuyển mô hình lên GPU (nếu có).
- Dự đoán và Ghi Kết quả:
    - Lặp qua `test_loader`, thực hiện dự đoán trên từng batch.
    - Tính xác suất của lớp mục tiêu bằng cách sử dụng hàm softmax và lấy phần bù của xác suất cho lớp "không có mã ẩn" (`1 - softmax`).
    - Lưu tên hình ảnh và xác suất dự đoán vào từ điển `result`.
- Tạo và Ghi Tệp `Submission`:
    - Chuyển đổi `result` thành DataFrame `submission`.
    - Ghi DataFrame vào tệp CSV `submission.csv` trong thư mục `OUTPUT_DIR`.

In [ ]:
'''
check log.txt
'''
try:
  with open("/kaggle/working/log.txt", 'r') as f:
    log_contents = f.read()
  print(f"The LOG variable contains: {log_contents}")
except FileNotFoundError:
  print("Error: The file /kaggle/working/log.txt doesn't exist.")

### check log:
- kiểm tra sự tồn tại của tệp log.txt trong thư mục /kaggle/working/, đọc và in nội dung của nó nếu tệp tồn tại. Nếu tệp không tồn tại, một thông báo lỗi sẽ được hiển thị.

In [ ]:
submiss_run()

## Kết quả huấn luyện mô hình:

### Kết quả nộp file trên Kaggle : 
- Link code đã nộp trên Kaggle: [Submission](https://www.kaggle.com/code/aleskab400/alaska2zero/notebook)
- Kết quả tốt nhất: 
    - Private Score: 0.753
    - Public Score: 0.776
- Nhận xét: 
    - Kết quả trên phụ thuộc vào tài nguyên sử dụng gồm: loại mô hình EfficientNet B2, số Epoch huấn luyện = 1
    - Để nâng cao kết quả, có thể áp dụng mô hình EfficientNet B4 trở lên, thêm số Epoch huấn luyện.

## Kết luận:


### Khó khăn gặp phải:
- Các mảng kiến thức mới về machine learning, các model huấn luyện là một thách thức lớn.
- Phần cứng cũng là một hạn chế cho việc thực hiện huấn luyện dữ liệu, mặc dù có thể sử dụng tài nguyên miễn phí trên Kaggle những với tập dữ liệu huấn luyện khá lớn (75.000 x 4 ) cũng dẫn đến việc mất nhiều thời gian huấn luyện.
- Một số khó khăn trong làm việc nhóm như quản lý thời gian và phân chia công việc.


### Kinh nghiệm và kiến thức đạt được :
- Có kiến thức về các phương pháp ẩn dữ liệu khác nhau, đặc biệt là việc ẩn dữ liệu liên quan đến việc ẩn dữ liệu trong các tệp ảnh JPEG.
- Mang đến cơ hội khám phá các ứng dụng của Machine Learning trong lĩnh vực An ninh mạng, trong việc phát hiện dữ liệu bí mật.
- Có thêm kinh nghiệm thực tế với các kỹ thuật xử lý hình ảnh, hiểu được cách các ảnh JPEG được cấu trúc và xử lý, cũng như cách dữ liệu có thể được nhúng vào chúng.
- Có kiến thức về Machine Learning, các kỹ thuật tiền xử lý, tạo model... 